In [16]:
from improved_apriori import Improved_Apriori
from efficient_apriori import apriori
import pandas as pd
import numpy as np


In [2]:
data = {'T1': ['I1', 'I2', 'I5', 'I3'],
        'T2': ['I2', 'I4', 'I3'],
        'T3': ['I2', 'I4'],
        'T4': ['I1', 'I2', 'I4'],
        'T5': ['I1', 'I3'],
        'T6': ['I2', 'I3'],
        'T7': ['I1', 'I3'],
        'T8': ['I1', 'I2', 'I3', 'I5'],
        'T9': ['I1', 'I2', 'I3']
        }

###  In this test, we check our own Apriori implementation against Efficient Apriori to ensure that our algorithm is correct

In [4]:
"""
To make things simpler,we start off with a known dataset and what we will do is to check the output of each steps between the two implementation

1. Pruning step
2. Candidate generation step
3. Compare output of the frequent itemset

"""
print("\n Own Apriori: \n")
improved_apriori = Improved_Apriori(data, min_support=0.001, min_confidence=0.1,verbose=2)
improved_apriori_itemset = improved_apriori.apriori()
l = list(data.values())

print("\n Efficient Apriori: \n")
efficient_apriori_itemsets, _ = apriori(l, min_support=0.001, min_confidence=1, verbosity=2)


 Own Apriori: 

Found 5 candidate itemsets from 1st Level
Found 5 frequent itemsets from 1th item candidate sets
Found 10 candidates for 2th item candidate sets
Time taken to find 2th item candidate sets: 2.3365020751953125e-05


100%|██████████| 10/10 [00:00<00:00, 76538.39it/s]


Found 9 frequent itemsets from 2th item candidate sets
Found 7 candidates for 3th item candidate sets
Time taken to find 3th item candidate sets: 5.7220458984375e-05


100%|██████████| 7/7 [00:00<00:00, 71610.07it/s]


Found 6 frequent itemsets from 3th item candidate sets
Found 1 candidates for 4th item candidate sets
Time taken to find 4th item candidate sets: 3.7670135498046875e-05


100%|██████████| 1/1 [00:00<00:00, 14364.05it/s]


Found 1 frequent itemsets from 4th item candidate sets
Found 0 candidates for 5th item candidate sets
Time taken to find 5th item candidate sets: 2.384185791015625e-05


0it [00:00, ?it/s]

Found 0 frequent itemsets from 5th item candidate sets

 Efficient Apriori: 

Generating itemsets.
 Counting itemsets of length 1.
  Found 5 candidate itemsets of length 1.
  Found 5 large itemsets of length 1.
    [('I1',), ('I2',), ('I5',), ('I3',), ('I4',)]
 Counting itemsets of length 2.
  Found 10 candidate itemsets of length 2.
   [('I1', 'I2'), ('I1', 'I3'), ('I1', 'I4'), ('I1', 'I5'), ('I2', 'I3'), ('I2', 'I4'), ('I2', 'I5'), ('I3', 'I4'), ('I3', 'I5'), ('I4', 'I5')]
    Iterating over transactions.
  Found 9 large itemsets of length 2.
   [('I1', 'I2'), ('I1', 'I3'), ('I1', 'I4'), ('I1', 'I5'), ('I2', 'I3'), ('I2', 'I4'), ('I2', 'I5'), ('I3', 'I4'), ('I3', 'I5')]
 Counting itemsets of length 3.
  Found 7 candidate itemsets of length 3.
   [('I1', 'I2', 'I3'), ('I1', 'I2', 'I4'), ('I1', 'I2', 'I5'), ('I1', 'I3', 'I4'), ('I1', 'I3', 'I5'), ('I2', 'I3', 'I4'), ('I2', 'I3', 'I5')]
    Iterating over transactions.
  Found 6 large itemsets of length 3.
   [('I1', 'I2', 'I3'), ('I1',

In [12]:
# Next we look at a slightly larger dataset and compare the output across different support
data = pd.read_csv('test_dataset/bread basket.csv')
data.drop_duplicates(inplace=True)

data.head(10)

,Transaction,Item,date_time,period_day,weekday_weekend
0,1,Bread,30-10-2016 09:58,morning,weekend
1,2,Scandinavian,30-10-2016 10:05,morning,weekend
3,3,Hot chocolate,30-10-2016 10:07,morning,weekend
4,3,Jam,30-10-2016 10:07,morning,weekend
5,3,Cookies,30-10-2016 10:07,morning,weekend
6,4,Muffin,30-10-2016 10:08,morning,weekend
7,5,Coffee,30-10-2016 10:13,morning,weekend
8,5,Pastry,30-10-2016 10:13,morning,weekend
9,5,Bread,30-10-2016 10:13,morning,weekend
10,6,Medialuna,30-10-2016 10:16,morning,weekend


In [13]:
data = data.groupby('Transaction')['Item'].apply(list)
data_dict = data.to_dict()

In [15]:
# Data Processing where we structure the data to fit our own implementation
# Create a dictionary where the key is the index and the value is a tuple of items

# To fit Efficient-Apriori implementation
data_list =  list(data_dict.values())

In [17]:
# Compare the two itemsets
def test_apriori(frequent_itemset_1, frequent_itemset_2):
    if(len(frequent_itemset_1)!=len(frequent_itemset_2)):
        return False
    
    for i in range(len(frequent_itemset_1)):
        sorted_dict1 = {tuple(sorted(k)): v for k, v in frequent_itemset_1[i+1].items()}
        sorted_dict2 = {tuple(sorted(k)): v for k, v in frequent_itemset_2[i+1].items()}
        if(sorted_dict1!=sorted_dict2):
            return False
        
    return True


In [18]:
# Create a list to hold the different reasult
diff_result = []
for min_support in np.arange(0.001, 0.3, 0.0001):
    improved_apriori = Improved_Apriori(data_dict, min_support=min_support)

    # Own implementation
    frequent_itemset_dict = improved_apriori.apriori()

    # Efficient Apriori
    frequent_itemset_dict_2, _ = apriori(data_list, min_support=min_support, min_confidence=1)
    if not test_apriori(frequent_itemset_dict, frequent_itemset_dict_2):
        diff_result.append([frequent_itemset_dict, frequent_itemset_dict_2, min_support])

100%|██████████| 59/59 [00:00<00:00, 4343.99it/s]
0it [00:00, ?it/s]
100%|██████████| 43/43 [00:00<00:00, 4277.67it/s]
0it [00:00, ?it/s]
100%|██████████| 36/36 [00:00<00:00, 4114.08it/s]
0it [00:00, ?it/s]
100%|██████████| 31/31 [00:00<00:00, 4055.25it/s]
0it [00:00, ?it/s]
100%|██████████| 80/80 [00:00<00:00, 5475.06it/s]
0it [00:00, ?it/s]
100%|██████████| 78/78 [00:00<00:00, 5625.87it/s]
0it [00:00, ?it/s]
100%|██████████| 78/78 [00:00<00:00, 5483.58it/s]
0it [00:00, ?it/s]
100%|██████████| 73/73 [00:00<00:00, 5636.47it/s]
0it [00:00, ?it/s]
100%|██████████| 73/73 [00:00<00:00, 5485.11it/s]
0it [00:00, ?it/s]
100%|██████████| 73/73 [00:00<00:00, 5492.09it/s]
0it [00:00, ?it/s]
100%|██████████| 70/70 [00:00<00:00, 5453.32it/s]
0it [00:00, ?it/s]
100%|██████████| 70/70 [00:00<00:00, 5419.20it/s]
0it [00:00, ?it/s]
100%|██████████| 70/70 [00:00<00:00, 5454.33it/s]
0it [00:00, ?it/s]
100%|██████████| 67/67 [00:00<00:00, 5385.45it/s]
0it [00:00, ?it/s]
100%|██████████| 63/63 [00:00<00:0

In [19]:
# Print the min_support values where results differ
for i in range(len(diff_result)):
    print("The results are different for these min_support values:", diff_result[i][2])